# Style Transfer

<img src="https://i0.wp.com/chelseatroy.com/wp-content/uploads/2018/12/neural_style_transfer.png?resize=768%2C311&ssl=1">

La idea de este trabajo final es reproducir el siguiente paper:

https://arxiv.org/pdf/1508.06576.pdf

El objetivo es transferir el estilo de una imagen dada a otra imagen distinta. 

Como hemos visto en clase, las primeras capas de una red convolucional se activan ante la presencia de ciertos patrones vinculados a detalles muy pequeños.

A medida que avanzamos en las distintas capas de una red neuronal convolucional, los filtros se van activando a medida que detectan patrones de formas cada vez mas complejos.

Lo que propone este paper es asignarle a la activación de las primeras capas de una red neuronal convolucional (por ejemplo VGG19) la definición del estilo y a la activación de las últimas capas de la red neuronal convolucional, la definición del contenido.

La idea de este paper es, a partir de dos imágenes (una que aporte el estilo y otra que aporte el contenido) analizar cómo es la activación de las primeras capas para la imagen que aporta el estilo y cómo es la activación de las últimas capas de la red convolucional para la imagen que aporta el contenido. A partir de esto se intentará sintetizar una imagen que active los filtros de las primeras capas que se activaron con la imagen que aporta el estilo y los filtros de las últimas capas que se activaron con la imagen que aporta el contenido.

A este procedimiento se lo denomina neural style transfer.

# En este trabajo se deberá leer el paper mencionado y en base a ello, entender la implementación que se muestra a continuación y contestar preguntas sobre la misma.

# Una metodología posible es hacer una lectura rápida del paper (aunque esto signifique no entender algunos detalles del mismo) y luego ir analizando el código y respondiendo las preguntas. A medida que se planteen las preguntas, volviendo a leer secciones específicas del paper terminará de entender los detalles que pudieran haber quedado pendientes.

Lo primero que haremos es cargar dos imágenes, una que aporte el estilo y otra que aporte el contenido. A tal fin utilizaremos imágenes disponibles en la web.

In [1]:
# Imagen para estilo
!wget https://upload.wikimedia.org/wikipedia/commons/5/52/La_noche_estrellada1.jpg

# Imagen para contenido
!wget https://upload.wikimedia.org/wikipedia/commons/thumb/f/f4/Neckarfront_T%C3%BCbingen_Mai_2017.jpg/775px-Neckarfront_T%C3%BCbingen_Mai_2017.jpg

# Creamos el directorio para los archivos de salida
!mkdir /content/output

--2020-02-21 02:20:49--  https://upload.wikimedia.org/wikipedia/commons/5/52/La_noche_estrellada1.jpg
Resolving upload.wikimedia.org (upload.wikimedia.org)... 103.102.166.240, 2001:df2:e500:ed1a::2:b
Connecting to upload.wikimedia.org (upload.wikimedia.org)|103.102.166.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 223725 (218K) [image/jpeg]
Saving to: ‘La_noche_estrellada1.jpg’

La_noche_estrellada 100%[===================>] 218.48K  --.-KB/s    in 0.03s   

2020-02-21 02:20:49 (6.51 MB/s) - ‘La_noche_estrellada1.jpg’ saved [223725/223725]

--2020-02-21 02:20:51--  https://upload.wikimedia.org/wikipedia/commons/thumb/f/f4/Neckarfront_T%C3%BCbingen_Mai_2017.jpg/775px-Neckarfront_T%C3%BCbingen_Mai_2017.jpg
Resolving upload.wikimedia.org (upload.wikimedia.org)... 103.102.166.240, 2001:df2:e500:ed1a::2:b
Connecting to upload.wikimedia.org (upload.wikimedia.org)|103.102.166.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 153015 (

In [2]:
from keras.preprocessing.image import load_img, save_img, img_to_array
import numpy as np
from scipy.optimize import fmin_l_bfgs_b
import time
import argparse

from keras.applications import vgg19
from keras import backend as K
from pathlib import Path

Using TensorFlow backend.


In [0]:
# Definimos las imagenes que vamos a utilizar, y el directorio de salida

base_image_path = Path("/content/775px-Neckarfront_Tübingen_Mai_2017.jpg")
style_reference_image_path = Path("/content/La_noche_estrellada1.jpg")
result_prefix = Path("/content/output")

iterations = 100

# 1) En base a lo visto en el paper ¿Qué significan los parámetros definidos en la siguiente celda?

Respuesta:

*content_weight* corresponde a alfa, *style_weight* a beta.

Ambos representan la ponderación relativa en la loss de optimizacion del "error" de contenido y de estilo de la nueva imagen creada, respecto a lo decodificado por la VGG (pre-entrenada) sobre las imagenes de entrada

*Total_variation_weight* no se encuentra en el paper original, pero según la implementacion de la notebook, corresponde a una penalidad tipo L2 sobre la diferencia entre pixels adyacentes de la nueva imagen creada: [(imagen creada - shift de 1 pixel ver).^2 + (imagen creada - shift 1 pixel hor).^2].^1.25, y todo eso sumado pixel a pixel en los tres canales.

No es un Learing Rate porque no disminuye el update por gradiente, ni Regularización porque no actua sobre los pesos. Simplemente penaliza cambios bruscos entre pixels cercanos. 

In [0]:
content_weight = 1    #(alpha)
style_weight = 10   #(beta)
ratio = content_weight/style_weight

total_variation_weight = 0.1

In [0]:
# Definimos el tamaño de las imágenes a utilizar
width, height = load_img(base_image_path).size
img_nrows = 400
img_ncols = int(width * img_nrows / height)

# 2) Explicar qué hace la siguiente celda. En especial las últimas dos líneas de la función antes del return. ¿Por qué?

Ayuda: https://keras.io/applications/

Respuesta:

carga la imagen, en rgb

luego transforma el objeto en un array de img_nrows x img_ncols x 3(rgb)

expande a un array de 1 x img_nrows x img_ncols x 3(rgb). Esto es necesario porque VGG espera un batch de imagenes a procesar

pasa la imagen por un "acomodador" de niveles de color, para que sean compatibles con los usados para el entrenamiento previo de VGG.

In [0]:
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_nrows, img_ncols))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img

# 3) Habiendo comprendido lo que hace la celda anterior, explique de manera muy concisa qué hace la siguiente celda. ¿Qué relación tiene con la celda anterior?

Respuesta:

rearmando el array del flatten (en el optimizador) al formato/tamaño inicial, 

luego trabaja sobre los brillos: está sumando valores medios, inviertiendo el orden de los colores y clippeando los brillos en el rango 0-255

Basicamente "preprocess_input" a la inversa...

In [0]:
def deprocess_image(x):
    x = x.reshape((img_nrows, img_ncols, 3))
    # Remove zero-center by mean pixel
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [0]:
# get tensor representations of our images
# K.variable convierte un numpy array en un tensor, para 
base_image = K.variable(preprocess_image(base_image_path))
style_reference_image = K.variable(preprocess_image(style_reference_image_path))

In [9]:
combination_image = K.placeholder((1, img_nrows, img_ncols, 3))

Aclaración:

La siguiente celda sirve para procesar las tres imagenes (contenido, estilo y salida) en un solo batch.

In [0]:
# combine the 3 images into a single Keras tensor
input_tensor = K.concatenate([base_image,
                              style_reference_image,
                              combination_image], axis=0)

In [11]:
# build the VGG19 network with our 3 images as input
# the model will be loaded with pre-trained ImageNet weights
model = vgg19.VGG19(input_tensor=input_tensor,
                    weights='imagenet', include_top=False)
print('Model loaded.')

# get the symbolic outputs of each "key" layer (we gave them unique names).
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])




80142336/80134624 [==============================] - 7s 0us/step






Model loaded.


In [12]:
outputs_dict['block4_conv2'][0, :, :, :]

<tf.Tensor 'strided_slice:0' shape=(50, 64, 512) dtype=float32>

# 4) En la siguientes celdas:

- ¿Qué es la matriz de Gram?¿Para qué se usa?
- ¿Por qué se permutan las dimensiones de x?

Respuesta:

La *matrix Gram* es una matriz de (auto)correlación de features=respuesta de filtros, en una imagen feed_forward por la red.
"It consists of the correlations between the different filter responses
over the spatial extent of the feature maps"

La uso para ir alterando la imagen generada hasta que minimice(por gradientes) la diferencia con los features obtenidos a partir de la imagen que aporta el estilo. 
Voy alterando la imagen inicial 'ruido_blanco' para que active la red de la misma forma que la imagen 'style'

Se *permutan* porque la sumatoria del paper es sobre el maxpooling (averagepool debiera ser en realidad!) de cada filtro pasado sobre todo el layer que le corresponde.
Entonces el flatten me deja un shape('n_filtros' , img_nrows*img_ncols/2^nro_pooling), listo para hacer la sumatoria del paper con el producto escalar.


In [0]:
def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

# 5) Losses:

Explicar qué mide cada una de las losses en las siguientes tres celdas.

Rta:

*Content_loss:* corresponde a la diferencia de activación en feed_forward en un layer particular de la VGG (o sea ya features), entre la imagen base -que aporta el contenido- y la imagen nueva generada.

*Style_loss:* corresponde a la suma de 5 diferencias de activación en feed_forward (o sea ya features) contra 5 layers particulares de la VGG, esta vez entre la imagen que aporta el estilo y la imagen nueva generada. La loss está escalada por el nro_filtros^2 y el tamaño^2 (pero no exactamente como dice el paper)

*Total_variation_loss:* no se encuentra en el paper original, pero según la implementacion de la notebook, corresponde a una penalidad tipo L2 sobre la diferencia entre pixels adyacentes de la nueva imagen creada: [(imagen creada - shift de 1 pixel ver).^2 + (imagen creada - shift 1 pixel hor).^2].^1.25, y todo eso sumado pixel a pixel en los tres canales.

No es un Learing Rate porque no disminuye el update por gradiente, ni Regularización porque no actua sobre los pesos. Simplemente penaliza cambios bruscos entre pixels adyacentes, y no trabaja con los features!


In [0]:
def style_loss(style, combination):
    assert K.ndim(style) == 3
    assert K.ndim(combination) == 3
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_nrows * img_ncols
    return K.sum(K.square(S - C)) / (4.0 * (channels ** 2) * (size ** 2))

In [0]:
def content_loss(base, combination):
    return K.sum(K.square(combination - base))


In [0]:
def total_variation_loss(x):
    assert K.ndim(x) == 4
    a = K.square(
        x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, 1:, :img_ncols - 1, :])
    b = K.square(
        x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, :img_nrows - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))


In [0]:
# Armamos la loss total
loss = K.variable(0.0)
layer_features = outputs_dict['block5_conv2']
base_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss = loss + content_weight * content_loss(base_image_features,
                                            combination_features)

feature_layers = ['block1_conv1', 'block2_conv1',
                  'block3_conv1', 'block4_conv1',
                  'block5_conv1']
for layer_name in feature_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1, :, :, :] 
    combination_features = layer_features[2, :, :, :]
    sl = style_loss(style_reference_features, combination_features)
    loss = loss + (style_weight / len(feature_layers)) * sl
loss = loss + total_variation_weight * total_variation_loss(combination_image)

In [18]:
grads = K.gradients(loss, combination_image)

outputs = [loss]
if isinstance(grads, (list, tuple)):
    outputs += grads
else:
    outputs.append(grads)

f_outputs = K.function([combination_image], outputs)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


# 6) Explique el propósito de las siguientes tres celdas. ¿Qué hace la función fmin_l_bfgs_b? ¿En qué se diferencia con la implementación del paper? ¿Se puede utilizar alguna alternativa?

Respuesta:

*En la celda anteror computamos el gradiente de la loss respecto de los pixels de la combination_image que se va generando en la red. También estamos generando una serie de tensores y placeholders para los gradientes y la loss

*Luego en la primera celda, definimos una funcion que manipula los gradientes y la loss para dejarlos en el formato correcto

*Por ultimo generamos una clase que entrega 2 funciones 'callable' de loss y gradiente, como lo requiere el optimizador empleado. Se hace esto de la clase simplemente porque se supone (no tengo herramientas para verificar esto) que es mas rápido computacionalmente.

En cuanto al minimizador/optimizador de loss, el paper no indica nada sobre cual usar. Finaliza la explicación con la expresión de la loss. Como aclaramos antes, la loss de la notebook incorpora un término para suavizar pixels adyacentes que no es parte del trabajo original.
También encontré algunas diferencias en el cálculo de style_loss, sobre como se obtiene el nro_filtros y el tamaño, que no se hace correctamente aqui.

Pero la principal diferencia es que en esta implementacion **NO** estamos partiendo de una imagen ruido blanco, sino de la imagen base para hacer el grad descent.

Justamente *fmin_l_bfgs_b* es el minimizador por gradientes multivariable que se ocupa de ir alterando la imagen compuesta para reducir la loss.
Es un linesearch mutivariable, optimizado para trabajar con menos memoria. La última b proviene de una variante que admite poner constraints a los valores de cada varaible, en este caso sería la luminosidad del pixel. NO se está usando esta capacidad.

Solo podríamos utilizar el tipo de optimizadores visto en el curso (Adam/RMSProp, Adamax) si aceptaramos usar los gradientes aproximados, ya que para esta loss en particular, no van a saber como obtenerlos. Por eso el cálculo externo con la funcion K.gradients.

In [0]:
def eval_loss_and_grads(x):
    x = x.reshape((1, img_nrows, img_ncols, 3))
    outs = f_outputs([x])
    loss_value = outs[0]
    if len(outs[1:]) == 1:
        grad_values = outs[1].flatten().astype('float64')
    else:
        grad_values = np.array(outs[1:]).flatten().astype('float64')
    return loss_value, grad_values

# this Evaluator class makes it possible
# to compute loss and gradients in one pass
# while retrieving them via two separate functions,
# "loss" and "grads". This is done because scipy.optimize
# requires separate functions for loss and gradients,
# but computing them separately would be inefficient.

In [0]:
class Evaluator(object):

    def __init__(self):
        self.loss_value = None
        self.grads_values = None

    def loss(self, x):
        assert self.loss_value is None
        loss_value, grad_values = eval_loss_and_grads(x)
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

# 7) Ejecute la siguiente celda y observe las imágenes de salida en cada iteración.

In [21]:
evaluator = Evaluator()

# run scipy-based optimization (L-BFGS) over the pixels of the generated image
# so as to minimize the neural style loss
x = preprocess_image(base_image_path)

for i in range(iterations):
    print('Start of iteration', i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(),
                                     fprime=evaluator.grads, maxfun=20)
    print('Current loss value:', min_val)
    # save current generated image
    img = deprocess_image(x.copy())
    fname = result_prefix / ('output_at_iteration_%d.png' % i)
    save_img(fname, img)
    end_time = time.time()
    print('Image saved as', fname)
    print('Iteration %d completed in %ds' % (i, end_time - start_time))

Start of iteration 0
Current loss value: 13226607000.0
Image saved as /content/output/output_at_iteration_0.png
Iteration 0 completed in 15s
Start of iteration 1
Current loss value: 6070406700.0
Image saved as /content/output/output_at_iteration_1.png
Iteration 1 completed in 6s
Start of iteration 2
Current loss value: 4092913700.0
Image saved as /content/output/output_at_iteration_2.png
Iteration 2 completed in 6s
Start of iteration 3
Current loss value: 3241331700.0
Image saved as /content/output/output_at_iteration_3.png
Iteration 3 completed in 6s
Start of iteration 4
Current loss value: 2671764500.0
Image saved as /content/output/output_at_iteration_4.png
Iteration 4 completed in 6s
Start of iteration 5
Current loss value: 2276616200.0
Image saved as /content/output/output_at_iteration_5.png
Iteration 5 completed in 6s
Start of iteration 6
Current loss value: 2014607600.0
Image saved as /content/output/output_at_iteration_6.png
Iteration 6 completed in 6s
Start of iteration 7
Curr

# 8) Generar imágenes para distintas combinaciones de pesos de las losses. Explicar las diferencias. (Adjuntar las imágenes generadas como archivos separados.)

Respuesta:

In [0]:
content_weight = 1    #(alpha)
style_weight = 1000   #(beta)  10,100,1000
ratio = content_weight/style_weight

total_variation_weight = 0     #0.1

In [23]:
# build the VGG19 network with our 3 images as input
# the model will be loaded with pre-trained ImageNet weights
model = vgg19.VGG19(input_tensor=input_tensor,
                    weights='imagenet', include_top=False)
print('Model loaded.')

Model loaded.


In [24]:
evaluator = Evaluator()

# run scipy-based optimization (L-BFGS) over the pixels of the generated image
# so as to minimize the neural style loss
x = preprocess_image(base_image_path)

for i in range(iterations):
    #print('Start of iteration', i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(),
                                     fprime=evaluator.grads, maxfun=20)
    print('Current loss value:', min_val)
    # save current generated image
    img = deprocess_image(x.copy())
    fname = result_prefix / ('2output_at_iteration_%d.png' % i)
    save_img(fname, img)
    end_time = time.time()
    #print('Image saved as', fname)
    print('Iteration %d completed in %ds' % (i, end_time - start_time))

Current loss value: 13272767000.0
Iteration 0 completed in 6s
Current loss value: 6531764700.0
Iteration 1 completed in 6s
Current loss value: 4403015700.0
Iteration 2 completed in 6s
Current loss value: 3410093300.0
Iteration 3 completed in 6s
Current loss value: 2825404000.0
Iteration 4 completed in 6s
Current loss value: 2479296800.0
Iteration 5 completed in 6s
Current loss value: 2123625000.0
Iteration 6 completed in 6s
Current loss value: 1903600600.0
Iteration 7 completed in 6s
Current loss value: 1776778200.0
Iteration 8 completed in 6s
Current loss value: 1687114900.0
Iteration 9 completed in 6s
Current loss value: 1609122300.0
Iteration 10 completed in 6s
Current loss value: 1547281700.0
Iteration 11 completed in 6s
Current loss value: 1501555200.0
Iteration 12 completed in 6s
Current loss value: 1431520900.0
Iteration 13 completed in 6s
Current loss value: 1379914800.0
Iteration 14 completed in 6s
Current loss value: 1336122100.0
Iteration 15 completed in 6s
Current loss valu

# 9) Cambiar las imágenes de contenido y estilo por unas elegidas por usted. Adjuntar el resultado.

Respuesta: